In [1]:
jupyter_pwd = %pwd
if jupyter_pwd == "/":
    %cd /workspace


# ipynb形式のライブラリのインポート
%run ./lib/lib.ipynb

# ipynb形式のライブラリノートを.py形式に変更したものをインポート
import lib
import lib.lab_lib
from lib.lab_lib import *

# 生データの入ったCSVファイルの保持されたディレクトリ名を格納している変数
csvDirPath = "./csv_files/"

# NPBのベンチマーク名のリスト
benchmarkNames = ["cg", "ep", "ft", "is", "lu", "mg"]

# NPBのプロセス数
npb_process :list[int] = [2, 4, 8, 16, 32, 64, 128, 256]
train_npb_process :list[int] = npb_process[:-1]
test_npb_process :list[int] = npb_process[-1:]
# NPBのCGの初期変数
cg_na: list[int] = [14000, 30000, 75000, 100000, 1500000]
cg_nonzer: list[int] = [11, 12, 13, 14, 15, 18, 21]
cg_niter: list[int] = [15, 30, 75, 90, 100]
cg_shift: list[int] = [20, 40, 60, 80, 110, 200]

train_cg_na: list[int] = cg_na[:-1]
train_cg_nonzer: list[int] = cg_nonzer[:-1]
train_cg_niter: list[int] = cg_niter[:-1]
train_cg_shift: list[int] = cg_shift[:-1]

test_cg_na: list[int] = cg_na[-1:]
test_cg_nonzer: list[int] = cg_nonzer[-1:]
test_cg_niter: list[int] = cg_niter[-1:]
test_cg_shift: list[int] = cg_shift[-1:]

# LULESH ベンチマークプログラムのプロセス数・問題サイズ・イテレーション数
lulesh_processes: list[int] = [8, 27, 64, 125, 216, 343, 512]
lulesh_iterations: list[int] = [8, 16, 32, 64, 128, 256]
lulesh_sizes: list[int] = [16, 24, 32, 48, 64, 128]

train_lulesh_processes: list[int] = [8, 27, 64, 125, 216, 343]
train_lulesh_iterations: list[int] = [8, 16, 32, 64, 128]
train_lulesh_sizes: list[int] = [16, 24, 32, 48]

test_lulesh_processes: list[int] = [512, 729, 1000]
test_lulesh_iterations: list[int] = [256, 512, 1024]
test_lulesh_sizes: list[int] = [64, 96, 128]


list_modelName: list[str] = [
    "modelIp",
    "modelLog",
    "modelLinAndIp",
    "modelLinAndLog",
    "modelIpAndLin",
    "modelIpAndLog",
    "modelLogAndLin",
    "modelLogAndIp",
    "modelProcessDividedByProblemSize",
    "modelProblemSizeDividedByProcess",
    "modelInfiniteProductOfProblemSizeMultipliedByProcesses",
    "modelInfiniteProductOfProblemSizeDividedByProcesses",
    "modelLinearSumOf2elementCombination",
    "modelLinearSumOfElementCombinations",
    "modelLinearSumOf2elementCombinationWithSquared",
    "modelLinearSumOf2elementCombinationWithCubed",
    "modelSquareRootOfProcess",
    "modelSquareRootTimesOtherElems",
    "modelObeyOneParameter",
    "modelLin"
    # "modelBasicTree",
]
list_csvDir = [
    "./csv_files/lulesh_1st/",
    "./csv_files/lulesh_2nd/",
    "./csv_files/lulesh_3rd/",
]

DEBUG:__main__:hello
DEBUG:lib.lab_lib:hello


# モデル構築@CG


In [2]:
train_cg_nonzer

[11, 12, 13, 14, 15, 18]

In [3]:
expVar: list[str] = ["process", "na", "nonzer", "niter", "shift"]
resVar: str = "#Call"

trainDF_cg: pd.DataFrame = return_rawDF_cg(
    list_process=train_npb_process,
    list_na=train_cg_na,
    list_nonzer=train_cg_nonzer,
    list_niter=train_cg_niter,
    list_shift=train_cg_shift,
    csvDir=csvDirPath,
)

trainDF_cg["functionName"] = trainDF_cg["Name"]

/root/src/lib/lab_lib.py:8785: UserWarning: ./csv_files/cg_na75000_nonzer18_niter30_shift60_process4.csv is empty.
  warnings.warn(f"{filePath} is empty.")
/root/src/lib/lab_lib.py:8785: UserWarning: ./csv_files/cg_na75000_nonzer15_niter90_shift80_process8.csv is empty.
  warnings.warn(f"{filePath} is empty.")


In [4]:
testDF_cg: pd.DataFrame = return_rawDF_cg(
    list_process=test_npb_process,
    list_na=test_cg_na,
    list_nonzer=test_cg_nonzer,
    list_niter=test_cg_niter,
    list_shift=test_cg_shift,
    csvDir=csvDirPath,
)
testDF_cg["functionName"] = testDF_cg["Name"]

In [5]:
dict_model_cg: dict[str, dict[str, any]] = {}
functionNames_cg = sorted(list(set(trainDF_cg["Name"])))

for functionName in functionNames_cg:
    trainDF_cg_perFunc: pd.DataFrame = trainDF_cg[trainDF_cg["Name"] == functionName]
    bestModelDict: dict = return_bestModelObject(
        inputDF=trainDF_cg_perFunc,
        list_expVar=expVar,
        list_resVar=[resVar],
        list_modelName=list_modelName,
    )
    bestModel = bestModelDict["object"]

    dict_model_perFunc: dict[str, any] = {
        "call_perFunc": bestModel,
    }
    dict_model_cg[functionName] = dict_model_perFunc

/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estima

In [6]:
for i, sr in trainDF_cg.reset_index().iterrows():
    if i == 1:
        print(pd.DataFrame(sr).T)

  index  %Time Exclusive Inclusive #Call #Subrs Name process     na nonzer  \
1     1  100.0         1     7,550   1.0   85.5   CG       2  14000     11   

  niter shift functionName  
1    15    20           CG  


In [7]:
_list_series: list[pd.Series] = []
for i, sr in trainDF_cg.iterrows():
    functionName = sr["Name"]
    DF_to_predict: pd.DataFrame = pd.DataFrame(sr).T[expVar]
    DF_to_predict = DF_to_predict.astype(
        {
            "process": "int32",
            "na": "int32",
            "nonzer": "int32",
            "niter": "int32",
            "shift": "int32",
        }
    )

    predicted_call: float = float(
        dict_model_cg[functionName]["call_perFunc"].predict(inputDF=DF_to_predict)
    )
    real_call: float = sr["#Call"]
    _list_series.append(
        pd.Series(
            {
                "process": sr["process"],
                "na": sr["na"],
                "nonzer": sr["nonzer"],
                "niter": sr["niter"],
                "shift": sr["shift"],
                "predicted_call": predicted_call,
                "real_call": real_call,
            }
        )
    )
DF_cg_about_call: pd.DataFrame = pd.DataFrame(data=_list_series)

In [8]:
DF_cg_about_call

,process,na,nonzer,niter,shift,predicted_call,real_call
0,2.0,14000.0,11.0,15.0,20.0,1.0,1.0
1,2.0,14000.0,11.0,15.0,20.0,1.0,1.0
2,2.0,14000.0,11.0,15.0,20.0,1.0,1.0
3,2.0,14000.0,11.0,15.0,20.0,1.0,1.0
4,2.0,14000.0,11.0,15.0,20.0,1.0,1.0
...,...,...,...,...,...,...,...
100735,128.0,100000.0,18.0,90.0,110.0,1.0,1.0
100736,128.0,100000.0,18.0,90.0,110.0,1.0,1.0
100737,128.0,100000.0,18.0,90.0,110.0,1.0,1.0
100738,128.0,100000.0,18.0,90.0,110.0,1.0,1.0


In [9]:
DF_cg_about_call = add_relativeErrorRateCol(
    inputDF=DF_cg_about_call,
    real_colName="real_call",
    predicted_colName="predicted_call",
    targetColName="relative_error_rate",
)
DF_cg_about_call

,process,na,nonzer,niter,shift,predicted_call,real_call,relative_error_rate
0,2.0,14000.0,11.0,15.0,20.0,1.0,1.0,0.0
1,2.0,14000.0,11.0,15.0,20.0,1.0,1.0,0.0
2,2.0,14000.0,11.0,15.0,20.0,1.0,1.0,0.0
3,2.0,14000.0,11.0,15.0,20.0,1.0,1.0,0.0
4,2.0,14000.0,11.0,15.0,20.0,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...
100735,128.0,100000.0,18.0,90.0,110.0,1.0,1.0,0.0
100736,128.0,100000.0,18.0,90.0,110.0,1.0,1.0,0.0
100737,128.0,100000.0,18.0,90.0,110.0,1.0,1.0,0.0
100738,128.0,100000.0,18.0,90.0,110.0,1.0,1.0,0.0


In [10]:
DF_cg_about_call.mean()

process                    36.303752
na                      54737.939250
nonzer                     13.831745
niter                      52.495533
shift                      61.995235
predicted_call         112450.929589
real_call              112450.929589
relative_error_rate         2.096187
dtype: float64

In [11]:
_list_series: list[pd.Series] = []
for i, sr in testDF_cg.iterrows():
    functionName = sr["Name"]
    DF_to_predict: pd.DataFrame = pd.DataFrame(sr).T[expVar]
    DF_to_predict = DF_to_predict.astype(
        {
            "process": "int32",
            "na": "int32",
            "nonzer": "int32",
            "niter": "int32",
            "shift": "int32",
        }
    )

    predicted_call: float = float(
        dict_model_cg[functionName]["call_perFunc"].predict(inputDF=DF_to_predict)
    )
    real_call: float = sr["#Call"]
    _list_series.append(
        pd.Series(
            {
                "process": sr["process"],
                "na": sr["na"],
                "nonzer": sr["nonzer"],
                "niter": sr["niter"],
                "shift": sr["shift"],
                "predicted_call": predicted_call,
                "real_call": real_call,
            }
        )
    )
DF_cg_about_call_atTestEnv: pd.DataFrame = pd.DataFrame(data=_list_series)
DF_cg_about_call_atTestEnv = add_relativeErrorRateCol(
    inputDF=DF_cg_about_call_atTestEnv,
    real_colName="real_call",
    predicted_colName="predicted_call",
    targetColName="relative_error_rate",
)

In [12]:
DF_cg_about_call_atTestEnv.mean()

process                2.560000e+02
na                     1.500000e+06
nonzer                 2.100000e+01
niter                  1.000000e+02
shift                  2.000000e+02
predicted_call         4.861059e+06
real_call              4.508182e+06
relative_error_rate    5.442789e+00
dtype: float64